In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
churn = pd.read_csv("/kaggle/input/churn-modelling/Churn_Modelling.csv")
data = churn.copy()
data.head()

In [ ]:
data.info()

In [ ]:
#gereksiz kolonlarin silinmesi
data.drop(["RowNumber","CustomerId","Surname"],axis=1,inplace=True)

In [ ]:
#betimsel istatistikler
data.describe().T

In [ ]:
#DataVisualization
import seaborn as sns
import matplotlib.pyplot as plt
#Boxplot
for column in data.columns[:-1]:
    sns.boxplot(x="Exited",y=column,data=data);
    plt.show()

In [ ]:
#Barplot
for column in data.columns[:-1]:
    sns.barplot(x="Exited",y=column,data=data)
    plt.show()

In [ ]:
#Violinplot
for column in data.columns[:-1]:
    sns.violinplot(x="Exited",y=column,data=data)
    plt.show()

In [ ]:
data["Geography"].value_counts()

In [ ]:
data["Gender"].value_counts()

In [ ]:
data.Age.mean()

In [ ]:
data.groupby("Gender")["Age"].mean()

In [ ]:
data.groupby("Geography")["Age"].mean()

In [ ]:
data.groupby("Geography")["CreditScore"].mean()

In [ ]:
data.groupby("Geography")["EstimatedSalary"].mean()

In [ ]:
#Outliers(Age)
sns.boxplot(x=data["Geography"],y=data["Age"]);

In [ ]:
data["Age"].value_counts().sort_values()

In [ ]:
data_age = data["Age"]

In [ ]:
data_age.shape

In [ ]:
#IQR
Q1 = data_age.quantile(0.25)
Q3 = data_age.quantile(0.75)
IQR = Q3 - Q1
alt_sinir = Q1 - 1.5*IQR
ust_sinir = Q3 + 1.5*IQR

In [ ]:
alt_sinir

In [ ]:
ust_sinir

In [ ]:
aykiri_deger = (data_age < alt_sinir )| (data_age > ust_sinir)

In [ ]:
aykiri_index = data_age[aykiri_deger].index

In [ ]:
data_age.drop(index=aykiri_index,axis=1,inplace=True)

In [ ]:
data_age.shape

In [ ]:
data["Age"].shape

In [ ]:
data["Age"] = data_age

In [ ]:
data["Age"].shape

In [ ]:
data.describe().T

In [ ]:
#korelasyon

corr_columns = data[["Age","CreditScore","Tenure","Balance","NumOfProducts","EstimatedSalary"]]
sns.set()
corr = corr_columns.corr()
ax = sns.heatmap(corr,annot=True)
plt.show()

In [ ]:
data.isnull().sum()

In [ ]:
data["Age"] = data["Age"].fillna(data["Age"].mean())

In [ ]:
data.isnull().sum()

In [ ]:
X = data.drop("Exited",axis=1)
y = data["Exited"]

from sklearn.preprocessing import LabelEncoder, StandardScaler
encoder = LabelEncoder()
X["Gender"] = encoder.fit_transform(X["Gender"])
X[["France","Spain","Germany"]] = pd.get_dummies(X["Geography"])
X.drop("Geography",axis=1,inplace=True)

In [ ]:
X

In [ ]:
#Normalization
norm = lambda x:(x-x.min()) / (x.max() - x.min())
transformCol = X[["Balance","CreditScore","EstimatedSalary"]]
X[["Balance","CreditScore","EstimatedSalary"]] = norm(transformCol)

In [ ]:
X

In [ ]:
#train-test
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,
                                                random_state=42)
pd.DataFrame({"TrainRowCount": [X_train.shape[0],y_train.shape[0]],
             "TestRowCount": [X_test.shape[0],y_test.shape[0]]},
            index=["X","Y"])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
algoritmalar = [LogisticRegression(),
               RandomForestClassifier(),
               DecisionTreeClassifier(),
               MLPClassifier(),
               GaussianNB(),
               XGBClassifier(),
               CatBoostClassifier()]

In [ ]:
model = []
acc_score = []
y_pred = []
for i in algoritmalar:
    model_ = i.fit(X_train,y_train)
    model.append(model_)
    pred = model_.predict(X_test)
    y_pred.append(pred)
    acc_score.append(accuracy_score(y_test,pred))

In [ ]:
model

In [ ]:
acc_score

In [ ]:
pd.DataFrame({"Algoritmalar": ["LogReg","RandFor","DecTree","MLP","GausNB","XGB","CatBo"],
              "Scores": acc_score})

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [ ]:
#cross_validation
models = []
models.append(("LogReg",LogisticRegression()))
models.append(("RandFor",RandomForestClassifier()))
models.append(("DecTree",DecisionTreeClassifier()))
models.append(("MLP",MLPClassifier()))
models.append(("GausNB",GaussianNB()))
models.append(("XGB",XGBClassifier()))
models.append(("CatBo",CatBoostClassifier()))

In [ ]:
from sklearn.model_selection import cross_val_score
names = []
result_mean = []
result_std = []
x,y = X.values,y.values.ravel()
for n,m in models:
    cv_r = cross_val_score(m,x,y,cv=10,verbose=False)
    result_mean.append(cv_r.mean())
    result_std.append(cv_r.std())
    names.append(n)

In [ ]:
pd.DataFrame({"Algoritmalar":names,
             "Accuracy Ortalama":result_mean,
             "Accuracy Standart Sapma": result_std})

In [ ]:
?CatBoostClassifier

In [ ]:
#GridSearchCV(CatBoost)
best_params = {'depth':[4,5,6,7,8,9],
              'learning_rate':[0.01,0.02,0.03,0.04],
              'iterations': [10,20,30,40,50]}

In [ ]:
from catboost import CatBoostClassifier
catboost = CatBoostClassifier()
catboost_tuned = GridSearchCV(catboost,best_params,cv=10,n_jobs=-1).fit(X_train,y_train,verbose=False)

In [ ]:
catboost_tuned.best_params_

In [ ]:
catboost_t = CatBoostClassifier(depth=8,iterations=50,learning_rate=0.04)
cb_tuned_model = catboost_t.fit(X_train,y_train,verbose=False)
y_pred = cb_tuned_model.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
importances_ = cb_tuned_model.feature_importances_
indices_ = np.argsort(importances_)
features = X.columns

plt.title("CatBoost Best Feature")
plt.barh(range(len(indices_)),
        importances_[indices_],align="center")
plt.yticks(range(len(indices_)),[features[i] for i in indices_])   
plt.show()

In [ ]:
cb_tuned_model.predict_proba(X_test)